<a href="https://colab.research.google.com/github/Manish-Singh-Mehra/Invoice_OCR/blob/main/Invoice_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2 as cv
import numpy as np

In [ ]:
image = cv.imread('/content/drive/MyDrive/Colab Notebooks/Sample_invoices/Permali Wallace Pvt Ltd-1.jpg')

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(image)

In [ ]:
base_image = image.copy()

In [ ]:
def noise_removal(book):
  kernel = np.ones((1,1),np.uint8)
  book = cv.dilate(book,kernel,iterations=1)
  book = cv.erode(book, kernel, iterations=1)
  book = cv.morphologyEx(book, cv.MORPH_CLOSE, kernel)
  book = cv.medianBlur(book,3)
  return (book)

In [ ]:
invert_image = cv.bitwise_not(image)
gray = cv.cvtColor(invert_image, cv.COLOR_BGR2GRAY)
thresh, bw = cv.threshold(gray, 27,255, cv.THRESH_BINARY)
cv2_imshow(bw)

In [ ]:
def remove_borders(cvImage):
  contours, heiarchy = cv.findContours(cvImage, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
  cntSorted = sorted(contours, key = lambda x: cv.contourArea(x))
  cnt = cntSorted[-1]
  x,y,w,h = cv.boundingRect(cnt)
  crop = image[y: y+h, x : x+w]
  return(crop)

In [ ]:
borderless = remove_borders(gray)

In [ ]:
def getSkwedAngle(cvImage)-> float:
  grey = cv.cvtColor(cvImage, cv.COLOR_BGR2GRAY)
  blur = cv.GaussianBlur(grey, (9,9), 0)
  thresh = cv.threshold(blur, 0,255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)[1]
  kernel = cv.getStructuringElement(cv.MORPH_RECT,(30,5))
  dilate = cv.dilate(thresh, kernel, iterations=2)
  contours, heiarchy = cv.findContours(dilate, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
  contours = sorted(contours, key = cv.contourArea, reverse = True)
  for c in contours:
    rect = cv.boundingRect(c)
    x,y,w,h = rect
    cv.rectangle(borderless, (x,y), (x+w,y+h), (0,255,0),2)
  largestcontour = contours[0]
  minAreaRect = cv.minAreaRect(largestcontour)
  angle = minAreaRect[-1]
  if angle<-45:
    angle = 90 + angle
  return (-1*angle)

In [ ]:
def rotateImage(cvImage, angle:float):
  newImage = cvImage.copy()
  (h,w) = newImage.shape[:2]
  center = (h//2, w//2)
  M = cv.getRotationMatrix2D(center, angle, 1)
  newImage = cv.warpAffine(newImage,M,(h,w), flags = cv.INTER_CUBIC, borderMode= cv.BORDER_REPLICATE)
  return newImage

In [ ]:
def deskew(cvImage):
  angle = getSkwedAngle(cvImage)
  return rotateImage(cvImage, angle )

In [ ]:
fixed = deskew(borderless)
cv2_imshow(fixed)